In [1]:
import Random
import LinearAlgebra as la
import ForwardDiff as fd
import ReverseDiff as rd
rng = Random.MersenneTwister();

using BenchmarkTools: @btime

In [44]:
include("layer.jl")
include("neuralnetwork.jl")

In [3]:
n = 10

10

In [4]:
positions = rand(rng, Float64, n);

In [47]:
@time nn = NeuralNetwork(input = positions, rng = rng,
    Dense(n), Sigmoid,
    Dense(n), Sigmoid,
    Dense(n), Sigmoid,
    Dense(n), Sigmoid,
    Dense(n), Sigmoid,
    Dense(n), Sigmoid,
    Dense(n), Sigmoid,
    Dense(n), Sigmoid,
    Dense(n), Sigmoid,
    Dense(n), Sigmoid,
    Dense(1), Exp,
    );

  0.150187 seconds (80.89 k allocations: 4.339 MiB, 99.44% compilation time)


In [51]:
@time model!(nn, positions);

  0.000024 seconds (1 allocation: 16 bytes)


In [53]:
@time gradient!(nn);

  0.000030 seconds (1 allocation: 224 bytes)


In [55]:
@time QF(nn);

  0.000010 seconds (1 allocation: 144 bytes)


In [49]:
@time kinetic(positions, nn);

  0.000041 seconds (1 allocation: 16 bytes)


In [41]:
@time nn_der = rd.gradient(x -> diffable_model(nn.layers, x)[1], positions);
2 * nn_der ./ nn.output[1] ≈ QF(nn)

  0.044378 seconds (20.41 k allocations: 1.171 MiB, 99.06% compilation time)


true

In [ ]:
params = []
gradient = []

for layer in nn.layers
    if typeof(layer) <: Dense
        push!(params, layer.W)
        push!(params, layer.b)
        
        push!(gradient, layer.W_g)
        push!(gradient, layer.b_g)
    end
end
params = Vector{Array{Float64}}(params)
gradient = Vector{Array{Float64}}(gradient);

In [ ]:
for param in params
    param .+= 0.1
end

In [57]:
col = rand(1000);

In [60]:
row = rand(1, 1000)

1×1000 Matrix{Float64}:
 0.824816  0.913507  0.553956  0.194236  …  0.0818118  0.565737  0.942

In [61]:
function test(v)
    return sum(v)
end

test (generic function with 2 methods)

In [68]:
@btime test(col)

  116.231 ns (1 allocation: 16 bytes)


504.32164691570364

In [71]:
@btime test(row)

  117.838 ns (1 allocation: 16 bytes)


504.7683862771142